In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import random
import time
import warnings; warnings.simplefilter('ignore')

In [2]:
def EnhancedCPM(eps,k,m):
    nv = 1
    for j in range(1,k+1):
        nv *= (m[j]+2)
        
    v = np.zeros((nv,k+1))
    index = np.zeros((k+1,int(max(m))+2,int(nv)))
    n = np.zeros((k+1,int(max(m))+2))
    for i in range(nv):
        I = i
        for j in range(k-1):
            v[i][k-j] = I%(m[k-j]+2)
            index[k-j][int(v[i][k-j])][int(n[k-j][int(v[i][k-j])])] = i
            n[k-j][int(v[i][k-j])] += 1
            I = int(I/(m[k-j]+2))
        v[i][1] = I
        index[1][I][int(n[1][I])] = i
        n[1][I] += 1
    #print(v)
    #print(n)
    #print(index)
    
    tild_sensitivity = np.zeros(k+1)
    for j in range(1,k+1):
        tild_sensitivity[j] = 2/(m[j]+1)
        
    b = np.zeros(k+1)
    for j in range(1,k+1):
        A = tild_sensitivity[j]*(math.exp(eps[j])*(math.exp(m[j]*eps[j])-1)/(math.exp(eps[j])-1))
        B = (1 + m[j]*tild_sensitivity[j])*math.exp((m[j]+1)*eps[j])
        U = eps[j]/math.log((A + math.sqrt(A**2 + 4*B))/2)
        b[j] = ((m[j]+3)/(m[j]+4))*U
        
    X = np.zeros(k+1)
    Y = np.zeros((k+1,max(m)+2))
    for j in range(1,k+1):
        q = (math.exp(eps[j]/b[j])/(2*((math.exp(eps[j]/b[j])+1)**2)*(math.exp(eps[j])-1)))*((math.exp(eps[j])-1)*(math.exp((m[j]+1)*eps[j])-1)-tild_sensitivity[j]*(math.exp(eps[j])/math.exp(eps[j]/b[j]))*(m[j]*math.exp((m[j]+1)*eps[j])-(m[j]+1)*math.exp(m[j]*eps[j])+1))
        X[j] = math.exp((m[j]+1)*eps[j])/(2*q*(math.exp(eps[j]/b[j])+1))
        for i in range(1,m[j]+2):
            Y[j][i] = (1/(2*q))*(math.exp(eps[j]/b[j])/(math.exp(eps[j]/b[j])+1))*(1+tild_sensitivity[j]*(math.exp(eps[j])/math.exp(eps[j]/b[j]))*((m[j]+1-i)*math.exp(m[j]*eps[j])-(math.exp(m[j]*eps[j])-1)/(math.exp(eps[j])-1)))
        
    d = np.zeros((k+1,max(m)+2))
    for j in range(1,k+1):
        d[j][0] = 2*X[j]
        for i in range(1,m[j]+1):
            d[j][i] = 2*(Y[j][i]-Y[j][i+1])
        d[j][m[j]+1] = 2*Y[j][m[j]+1]
    #print(d)
    
    N = np.ones(nv)
    for i in range(nv):
        for j in range(1,k+1):
            N[i] *= d[j][int(v[i][j])]
    #print(N)
    
    p0 = np.ones(int(nv)+1)*(1/int(nv))
    p0[int(nv)] = math.exp(np.sum(eps))
    fun = lambda p: p[int(nv)]
    
    cons = ()
    cons = cons + ({'type': 'eq', 'fun': lambda p: np.sum(p[i] for i in range(int(nv)))-1},)
    for i in range(1,k+1):
        cons = cons + ({'type': 'eq', 'fun': lambda p, i=i: np.sum(p[int(index[i][0][l])] for l in range(int(n[i][0]))) - 1/(math.exp(eps[i]/b[i])+1)},)
        for j in range(1,m[i]+1):
            cons = cons + ({'type': 'eq', 'fun': lambda p, i=i, j=j: np.sum(p[int(index[i][j][l])] for l in range(int(n[i][j]))) - tild_sensitivity[i]*(math.exp(j*eps[i])/(math.exp(eps[i]/b[i])+1))},)
    
    NV = nv
    NB = 0
    for i in range(1,k+1):
        NV = int(NV/(m[i]+2))
        NB += NV
        for j in range(int(nv)-NV):
            if v[j][i] != m[i]+1:
                cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: (p[j+NV]/N[j+NV]) - (p[j]/N[j])},)
    #print(NV)
    #print(NB)
    
    for j in range(int(nv)-NB):
        flag = 1
        for i in range(1,k+1):
            if v[j][i] == m[i]+1:
                flag = 0
        if flag == 1:
            cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: p[int(nv)] - (p[j+NB]/N[j+NB])/(p[j]/N[j])},)
    
    
    bnds = [(0,1)]*(int(nv)+1)
    bnds[int(nv)] = (1,math.exp(np.sum(eps)))
    
    res = minimize(fun, p0, bounds=bnds, constraints=cons)
    #print(math.log(res.fun))
    #print(res)
    
    return res.fun, res.x

In [3]:
def SubOptimal(eps,m,p_eps1):
    k = 2
    
    tild_sensitivity = np.zeros(k+1)
    for j in range(1,k+1):
        tild_sensitivity[j] = 2/(m[j]+1)
        
    b = np.zeros(k+1)
    for j in range(1,k+1):
        A = tild_sensitivity[j]*(math.exp(eps[j])*(math.exp(m[j]*eps[j])-1)/(math.exp(eps[j])-1))
        B = (1 + m[j]*tild_sensitivity[j])*math.exp((m[j]+1)*eps[j])
        U = eps[j]/math.log((A + math.sqrt(A**2 + 4*B))/2)
        b[j] = ((m[j]+3)/(m[j]+4))*U
        
    X = np.zeros(k+1)
    Y = np.zeros((k+1,max(m)+2))
    for j in range(1,k+1):
        q = (math.exp(eps[j]/b[j])/(2*((math.exp(eps[j]/b[j])+1)**2)*(math.exp(eps[j])-1)))*((math.exp(eps[j])-1)*(math.exp((m[j]+1)*eps[j])-1)-tild_sensitivity[j]*(math.exp(eps[j])/math.exp(eps[j]/b[j]))*(m[j]*math.exp((m[j]+1)*eps[j])-(m[j]+1)*math.exp(m[j]*eps[j])+1))
        X[j] = math.exp((m[j]+1)*eps[j])/(2*q*(math.exp(eps[j]/b[j])+1))
        for i in range(1,m[j]+2):
            Y[j][i] = (1/(2*q))*(math.exp(eps[j]/b[j])/(math.exp(eps[j]/b[j])+1))*(1+tild_sensitivity[j]*(math.exp(eps[j])/math.exp(eps[j]/b[j]))*((m[j]+1-i)*math.exp(m[j]*eps[j])-(math.exp(m[j]*eps[j])-1)/(math.exp(eps[j])-1)))
        
    d = np.zeros((k+1,max(m)+2))
    for j in range(1,k+1):
        d[j][0] = 2*X[j]
        for i in range(1,m[j]+1):
            d[j][i] = 2*(Y[j][i]-Y[j][i+1])
        d[j][m[j]+1] = 2*Y[j][m[j]+1]
    #print(d)
    
    N = np.zeros((m[1]+2,m[2]+2))
    for i in range(m[1]+2):
        for j in range(m[2]+2):
            N[i][j] = d[1][i]*d[2][j]
    #print(N)
    
    p0 = np.ones(2*m[1]+2*m[2]+5)*(1/(2*m[1]+2*m[2]+4))
    p0[2*m[1]+2*m[2]+4] = math.exp(p_eps1)
    fun = lambda p: p[2*m[1]+2*m[2]+4]
    
    cons = ()
    cons = cons + ({'type': 'eq', 'fun': lambda p: np.sum(p[i] for i in range(2*m[1]+2*m[2]+4)) + np.sum(p[m[1]+j]*(N[m[1]-i][j]/N[m[1]][j])*(1/math.exp(p_eps1*i)) for i in range(1,m[1]+1) for j in range(m[2])) - 1},)
    cons = cons + ({'type': 'eq', 'fun': lambda p: p[0] + p[m[1]+m[2]+1] + np.sum(p[m[1]+j]*(N[0][j]/N[m[1]][j])*(1/math.exp(p_eps1*m[1])) for j in range(m[2])) - 1/(math.exp(eps[1]/b[1])+1)},)
    for i in range(1,m[1]):
        cons = cons + ({'type': 'eq', 'fun': lambda p, i=i: p[i] + p[m[1]+m[2]+1+i] + np.sum(p[m[1]+j]*(N[i][j]/N[m[1]][j])*(1/math.exp(p_eps1*(m[1]-i))) for j in range(m[2])) - tild_sensitivity[1]*(math.exp(i*eps[1])/(math.exp(eps[1]/b[1])+1))},)
    cons = cons + ({'type': 'eq', 'fun': lambda p: np.sum(p[i] for i in range(m[1],m[1]+m[2]+1)) + p[2*m[1]+m[2]+1] - tild_sensitivity[1]*(math.exp(m[1]*eps[1])/(math.exp(eps[1]/b[1])+1))},)
    cons = cons + ({'type': 'eq', 'fun': lambda p: np.sum(p[m[1]]*(N[i][0]/N[m[1]][0])*(1/math.exp(p_eps1*(m[1]-i))) for i in range(m[1]+1)) + p[2*m[1]+m[2]+2] - 1/(math.exp(eps[2]/b[2])+1)},)
    for j in range(1,m[2]):
        cons = cons + ({'type': 'eq', 'fun': lambda p, j=j: np.sum(p[m[1]+j]*(N[i][j]/N[m[1]][j])*(1/math.exp(p_eps1*(m[1]-i))) for i in range(m[1]+1)) + p[2*m[1]+m[2]+2+j] - tild_sensitivity[2]*(math.exp(j*eps[2])/(math.exp(eps[2]/b[2])+1))},)
    cons = cons + ({'type': 'eq', 'fun': lambda p: np.sum(p[i] for i in range(m[1])) + p[m[1]+m[2]] + p[2*m[1]+2*m[2]+2] - tild_sensitivity[2]*(math.exp(m[2]*eps[2])/(math.exp(eps[2]/b[2])+1))},)
    
    for i in range(m[1]):
        cons = cons + ({'type': 'ineq', 'fun': lambda p, i=i: (p[i]/N[i][m[2]]) - (p[m[1]+m[2]-1]/N[m[1]][m[2]-1])*(1/math.exp(p_eps1*(m[1]-i)))},)
        cons = cons + ({'type': 'ineq', 'fun': lambda p, i=i: (p[m[1]+m[2]+1+i]/N[i][m[2]+1]) - (p[i]/N[i][m[2]])},)
    for i in range(m[1]-1):
        cons = cons + ({'type': 'ineq', 'fun': lambda p, i=i: (p[i+1]/N[i+1][m[2]]) - (p[i]/N[i][m[2]])},)
        cons = cons + ({'type': 'ineq', 'fun': lambda p, i=i: (p[m[1]+m[2]+2+i]/N[i+1][m[2]+1]) - (p[m[1]+m[2]+1+i]/N[i][m[2]+1])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: (p[m[1]+m[2]]/N[m[1]][m[2]]) - (p[m[1]-1]/N[m[1]-1][m[2]])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: (p[2*m[1]+m[2]+1]/N[m[1]][m[2]+1]) - (p[2*m[1]+m[2]]/N[m[1]-1][m[2]+1])},)
    for j in range(m[2]):
        cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: (p[m[1]+j+1]/N[m[1]][j+1]) - (p[m[1]+j]/N[m[1]][j])},)
        cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: (p[2*m[1]+m[2]+3+j]/N[m[1]+1][j+1]) - (p[2*m[1]+m[2]+2+j]/N[m[1]+1][j])},)
        cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: (p[2*m[1]+m[2]+2+j]/N[m[1]+1][j]) - (p[m[1]+j]/N[m[1]][j])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: (p[2*m[1]+m[2]+1]/N[m[1]][m[2]+1]) - (p[m[1]+m[2]]/N[m[1]][m[2]])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: (p[2*m[1]+2*m[2]+2]/N[m[1]+1][m[2]]) - (p[m[1]+m[2]]/N[m[1]][m[2]])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: (p[2*m[1]+2*m[2]+3]/N[m[1]+1][m[2]+1]) - (p[2*m[1]+m[2]+1]/N[m[1]][m[2]+1])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: (p[2*m[1]+2*m[2]+3]/N[m[1]+1][m[2]+1]) - (p[2*m[1]+2*m[2]+2]/N[m[1]+1][m[2]])},)
    
    for i in range(m[1]-1):
        cons = cons + ({'type': 'ineq', 'fun': lambda p, i=i: p[2*m[1]+2*m[2]+4] - math.exp(p_eps1*(m[1]-i))*((p[i+1]/N[i+1][m[2]])/(p[m[1]+m[2]-1]/N[m[1]][m[2]-1]))},)
        cons = cons + ({'type': 'ineq', 'fun': lambda p, i=i: p[2*m[1]+2*m[2]+4] - (p[m[1]+m[2]+2+i]/N[i+1][m[2]+1])/(p[i]/N[i][m[2]])},)
    for j in range(m[2]):
        cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: p[2*m[1]+2*m[2]+4] - math.exp(p_eps1)*((p[m[1]+j+1]/N[m[1]][j+1])/(p[m[1]+j]/N[m[1]][j]))},)
        cons = cons + ({'type': 'ineq', 'fun': lambda p, j=j: p[2*m[1]+2*m[2]+4] - (p[2*m[1]+m[2]+3+j]/N[m[1]+1][j+1])/(p[m[1]+j]/N[m[1]][j])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: p[2*m[1]+2*m[2]+4] - (p[2*m[1]+m[2]+1]/N[m[1]][m[2]+1])/(p[m[1]-1]/N[m[1]-1][m[2]])},)
    cons = cons + ({'type': 'ineq', 'fun': lambda p: p[2*m[1]+2*m[2]+4] - (p[2*m[1]+2*m[2]+3]/N[m[1]+1][m[2]+1])/(p[m[1]+m[2]]/N[m[1]][m[2]])},)
        
    
    bnds = [(0,1)]*(2*m[1]+2*m[2]+5)
    bnds[2*m[1]+2*m[2]+4] = (0,np.inf)
    
    res = minimize(fun, p0, bounds=bnds, constraints=cons)
    if res.message == 'Inequality constraints incompatible' or res.fun < math.exp(p_eps1):
        return np.sum(eps), [100,100]
    else:
        return res.fun, res.x

In [4]:
k = 3
r = 50
rt = np.zeros(2)

eps = np.zeros(4); m = np.zeros(4)
for j in range(r):
    for l in range(1,k+1):
        eps[l] = random.random()
    m = [0,1,1,1]
    start = time.time()
    EnhancedCPM(eps,k,m)
    end = time.time()
    rt[0] += end-start
    start = time.time()
    SubOptimal([0,eps[1],eps[2]],m,eps[1]*0.88)
    end = time.time()
    rt[1] += end-start
rt /= r

print(rt)

[0.55998127 0.02353025]
